# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fad2472dfa0>
├── 'a' --> tensor([[-0.2812, -1.9545, -0.6738],
│                   [ 1.1351,  1.4279,  0.2253]])
└── 'x' --> <FastTreeValue 0x7fad2472d6d0>
    └── 'c' --> tensor([[ 0.7742,  0.9259,  1.7446,  0.4975],
                        [-1.8115, -0.0306,  1.2756, -1.3858],
                        [ 1.5749,  0.6550,  0.0993, -0.6405]])

In [4]:
t.a

tensor([[-0.2812, -1.9545, -0.6738],
        [ 1.1351,  1.4279,  0.2253]])

In [5]:
%timeit t.a

65.2 ns ± 1.86 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fad2472dfa0>
├── 'a' --> tensor([[ 0.3956, -0.1852, -0.8122],
│                   [-0.2811, -0.1362,  1.0543]])
└── 'x' --> <FastTreeValue 0x7fad2472d6d0>
    └── 'c' --> tensor([[ 0.7742,  0.9259,  1.7446,  0.4975],
                        [-1.8115, -0.0306,  1.2756, -1.3858],
                        [ 1.5749,  0.6550,  0.0993, -0.6405]])

In [7]:
%timeit t.a = new_value

62 ns ± 3.8 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2812, -1.9545, -0.6738],
               [ 1.1351,  1.4279,  0.2253]]),
    x: Batch(
           c: tensor([[ 0.7742,  0.9259,  1.7446,  0.4975],
                      [-1.8115, -0.0306,  1.2756, -1.3858],
                      [ 1.5749,  0.6550,  0.0993, -0.6405]]),
       ),
)

In [10]:
b.a

tensor([[-0.2812, -1.9545, -0.6738],
        [ 1.1351,  1.4279,  0.2253]])

In [11]:
%timeit b.a

52.5 ns ± 1.47 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4963,  0.8287, -1.6400],
               [-0.1406,  0.3259, -1.3663]]),
    x: Batch(
           c: tensor([[ 0.7742,  0.9259,  1.7446,  0.4975],
                      [-1.8115, -0.0306,  1.2756, -1.3858],
                      [ 1.5749,  0.6550,  0.0993, -0.6405]]),
       ),
)

In [13]:
%timeit b.a = new_value

495 ns ± 5.37 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

834 ns ± 21 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 329 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

266 µs ± 20.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

223 µs ± 9.13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fac7caead30>
├── 'a' --> tensor([[[-0.2812, -1.9545, -0.6738],
│                    [ 1.1351,  1.4279,  0.2253]],
│           
│                   [[-0.2812, -1.9545, -0.6738],
│                    [ 1.1351,  1.4279,  0.2253]],
│           
│                   [[-0.2812, -1.9545, -0.6738],
│                    [ 1.1351,  1.4279,  0.2253]],
│           
│                   [[-0.2812, -1.9545, -0.6738],
│                    [ 1.1351,  1.4279,  0.2253]],
│           
│                   [[-0.2812, -1.9545, -0.6738],
│                    [ 1.1351,  1.4279,  0.2253]],
│           
│                   [[-0.2812, -1.9545, -0.6738],
│                    [ 1.1351,  1.4279,  0.2253]],
│           
│                   [[-0.2812, -1.9545, -0.6738],
│                    [ 1.1351,  1.4279,  0.2253]],
│           
│                   [[-0.2812, -1.9545, -0.6738],
│                    [ 1.1351,  1.4279,  0.2253]]])
└── 'x' --> <FastTreeValue 0x7fac7caeabb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.7 µs ± 2.68 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fac7caeae20>
├── 'a' --> tensor([[-0.2812, -1.9545, -0.6738],
│                   [ 1.1351,  1.4279,  0.2253],
│                   [-0.2812, -1.9545, -0.6738],
│                   [ 1.1351,  1.4279,  0.2253],
│                   [-0.2812, -1.9545, -0.6738],
│                   [ 1.1351,  1.4279,  0.2253],
│                   [-0.2812, -1.9545, -0.6738],
│                   [ 1.1351,  1.4279,  0.2253],
│                   [-0.2812, -1.9545, -0.6738],
│                   [ 1.1351,  1.4279,  0.2253],
│                   [-0.2812, -1.9545, -0.6738],
│                   [ 1.1351,  1.4279,  0.2253],
│                   [-0.2812, -1.9545, -0.6738],
│                   [ 1.1351,  1.4279,  0.2253],
│                   [-0.2812, -1.9545, -0.6738],
│                   [ 1.1351,  1.4279,  0.2253]])
└── 'x' --> <FastTreeValue 0x7fac7f049040>
    └── 'c' --> tensor([[ 0.7742,  0.9259,  1.7446,  0.4975],
                        [-1.8115, -0.0306,  1.2756, -1.3858],
                 

In [23]:
%timeit t_cat(trees)

34.4 µs ± 464 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

71.2 µs ± 4.75 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.2812, -1.9545, -0.6738],
                [ 1.1351,  1.4279,  0.2253]],
       
               [[-0.2812, -1.9545, -0.6738],
                [ 1.1351,  1.4279,  0.2253]],
       
               [[-0.2812, -1.9545, -0.6738],
                [ 1.1351,  1.4279,  0.2253]],
       
               [[-0.2812, -1.9545, -0.6738],
                [ 1.1351,  1.4279,  0.2253]],
       
               [[-0.2812, -1.9545, -0.6738],
                [ 1.1351,  1.4279,  0.2253]],
       
               [[-0.2812, -1.9545, -0.6738],
                [ 1.1351,  1.4279,  0.2253]],
       
               [[-0.2812, -1.9545, -0.6738],
                [ 1.1351,  1.4279,  0.2253]],
       
               [[-0.2812, -1.9545, -0.6738],
                [ 1.1351,  1.4279,  0.2253]]]),
    x: Batch(
           c: tensor([[[ 0.7742,  0.9259,  1.7446,  0.4975],
                       [-1.8115, -0.0306,  1.2756, -1.3858],
                       [ 1.5749,  0.6550,  0.0993, -0.6405]],
         

In [26]:
%timeit Batch.stack(batches)

85.3 µs ± 2.61 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.2812, -1.9545, -0.6738],
               [ 1.1351,  1.4279,  0.2253],
               [-0.2812, -1.9545, -0.6738],
               [ 1.1351,  1.4279,  0.2253],
               [-0.2812, -1.9545, -0.6738],
               [ 1.1351,  1.4279,  0.2253],
               [-0.2812, -1.9545, -0.6738],
               [ 1.1351,  1.4279,  0.2253],
               [-0.2812, -1.9545, -0.6738],
               [ 1.1351,  1.4279,  0.2253],
               [-0.2812, -1.9545, -0.6738],
               [ 1.1351,  1.4279,  0.2253],
               [-0.2812, -1.9545, -0.6738],
               [ 1.1351,  1.4279,  0.2253],
               [-0.2812, -1.9545, -0.6738],
               [ 1.1351,  1.4279,  0.2253]]),
    x: Batch(
           c: tensor([[ 0.7742,  0.9259,  1.7446,  0.4975],
                      [-1.8115, -0.0306,  1.2756, -1.3858],
                      [ 1.5749,  0.6550,  0.0993, -0.6405],
                      [ 0.7742,  0.9259,  1.7446,  0.4975],
                      [-1.8115, -

In [28]:
%timeit Batch.cat(batches)

157 µs ± 6.97 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

486 µs ± 42.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
